In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [55]:
##this is all wrong.  cross product between the words makes no sense lolol
#a sentence embedding tho.

class SimpleNN(nn.Module):
    def __init__(self
                 , vocab_size
                 , sequence_length
                 , hidden_dim
                 , word_embed
                 , sentence_embed
                 , balanced_dim
                ):
        super(SimpleNN, self).__init__()
        self.vocab_size = vocab_size
        self.sequence_length = sequence_length
        self.word_embed = word_embed
        
        self.word_embedding = nn.Linear(vocab_size,word_embed) #50 is word embeddings essentially
        self.sentence_embedding = nn.Linear(sequence_length,sentence_embed) #also could apply twice.
        self.we_down = nn.Linear(word_embed,balanced_dim)
        self.seq_down = nn.Linear(sentence_embed,balanced_dim) #could apply twice
    
        self.out = nn.Linear(balanced_dim**3,vocab_size)
        # Activation function

    def forward(self, x):
        print(x.shape)
        x = F.relu(self.word_embedding(x)) #sentence of word embeddings.  
        print(x.shape)
        x = x.permute(0, 2, 1)
        print(x.shape)
        x = F.relu(self.sentence_embedding(x))
        print(x.shape)
        x = x.permute(0, 2, 1)
        print(x.shape)
        # Flatten the sequence_length and vocab_size dimensions
        x = F.relu(torch.einsum('bij,bkm->bikj', x, x)) #interactions
        print(x.shape)
        x = F.relu(self.we_down(x))
        print(x.shape)
        x = x.permute(0, 3, 1, 2)
        print(x.shape)
        x = F.relu(self.seq_down(x))
        print(x.shape)
        x = x.permute(0, 1, 3, 2)
        print(x.shape)
        x = F.relu(self.seq_down(x)) #duplicated, could be a second layer.   
        print(x.shape)
        x = x.view(x.shape[0],-1)
        print(x.shape)
        # Pass through the second fully connected layer
        x = self.out(x)
        #x = F.softmax(x,dim=1)
        return x

In [56]:
# Hyperparameters
vocab_size = 3000  # Example vocab size
sequence_length = 1000  # Example sequence length
hidden_dim = 128  # Example hidden dimension
word_embed = 51
sentence_embed = 54
balanced_dim = 10
batch_size = 50

# Instantiate the model
model = SimpleNN(vocab_size, sequence_length, hidden_dim, word_embed,sentence_embed,balanced_dim)
print(model)

SimpleNN(
  (word_embedding): Linear(in_features=3000, out_features=51, bias=True)
  (sentence_embedding): Linear(in_features=1000, out_features=54, bias=True)
  (we_down): Linear(in_features=51, out_features=10, bias=True)
  (seq_down): Linear(in_features=54, out_features=10, bias=True)
  (out): Linear(in_features=1000, out_features=3000, bias=True)
)


In [57]:
rand_input = torch.rand(batch_size, sequence_length, vocab_size)
rand_input.shape

torch.Size([50, 1000, 3000])

In [58]:
model(rand_input).shape

torch.Size([50, 1000, 3000])
torch.Size([50, 1000, 51])
torch.Size([50, 51, 1000])
torch.Size([50, 51, 54])
torch.Size([50, 54, 51])
torch.Size([50, 54, 54, 51])
torch.Size([50, 54, 54, 10])
torch.Size([50, 10, 54, 54])
torch.Size([50, 10, 54, 10])
torch.Size([50, 10, 10, 54])
torch.Size([50, 10, 10, 10])
torch.Size([50, 1000])


torch.Size([50, 3000])

In [48]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Using the SimpleNN model define

In [49]:
total_params = count_parameters(model)
print(f"Total number of trainable parameters: {total_params}")

Total number of trainable parameters: 3211175
